<h1 style="font-size:30px;">Verifying the ASL Classifier for DPU compatibility</h1>  

In this notebook, we start with a model pre-trained for imagenet, ad verify if it is compatible with the DPU core.  This step is an important sanity check that should be performed BEFORE training.

**The proposed compatibility checking technique is summarized below:**

1. Instantiate our candidate model
2. Inspect model for Vitis-AI_DPU compatibility
3. Add our custom classifier layer for the ASL dataset
4. Inspect model for Vitis-AI_DPU compatibility
5. Iterate until a compatible model has been defined

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/VGG16_03_asl_vitis_ai.png' width=1000 align='center'><br/>

## Table of Contents
* [1 System Configuration](#1-System-Configuration)
* [2 Dataset and Training Configuration](#2-Dataset-and-Training-Configuration)
* [3 Inspecting the Original Model](#4-Inspecing-the-Original-Model)
* [4 Inspecting the Modified Model](#5-Inspecing-the-Modified-Model)
* [5 Conclusion](#6-Conclusion)

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import random
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
import zipfile
import requests
import glob as glob

from tensorflow.keras.utils import image_dataset_from_directory

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter)
from dataclasses import dataclass 

block_plot = False
plt.rcParams['figure.figsize'] = (12, 9)
SEED_VALUE = 42 

2023-04-24 10:50:16.614065: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 10:50:16.748555: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## 1 System Configuration

In [2]:
def system_config():
    
    # Get list of GPUs.
    gpu_devices = tf.config.list_physical_devices('GPU')
    print(gpu_devices)
    
    if len(gpu_devices) > 0:
        print('Using GPU')
        os.environ["CUDA_VISIBLE_DEVICES"] = '0'
        os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
        
        # If there are any gpu devices, use first gpu.
        tf.config.experimental.set_visible_devices(gpu_devices[0], 'GPU')
        
        # Grow the memory usage as it is needed by the process.
        tf.config.experimental.set_memory_growth(gpu_devices[0], True)
        
        # Enable using cudNN.
        os.environ['TF_USE_CUDNN'] = "true"
    else:
        print('Using CPU')

system_config()

[]
Using CPU


## 2 Dataset Configuration

In [3]:
@dataclass(frozen=True)
class DatasetConfig:
    NUM_CLASSES: int = 29
    IMG_HEIGHT:  int = 224
    IMG_WIDTH:   int = 224
    CHANNELS:    int = 3
    BATCH_SIZE:  int = 32
    DATA_ROOT:   str = './dataset_ASL_reduced'

## 3 Inspecting the Original Model

### 3.1 Create the original VGG-16 model

The Keras API provides the following utility that is used to instantiate a VGG-16 model, which can be initialized with either pre-trained weights from ImageNet or random initial weights. **In this notebook, we will load the model with weights from the ImageNet dataset** as a starting point for **Training.** For more information on the VGG-16 model available in Keras, refer to the: <a href="https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16" target="_blank">Keras VGG-16 Model API</a>

```python
tf.keras.applications.vgg16.VGG16(include_top=True, 
                                  weights='imagenet', 
                                  input_tensor=None,
                                  input_shape=None, 
                                  pooling=None, 
                                  classes=1000,
                                  classifier_activation='softmax',
                                 )
```

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/VGG16_01_imagenet.png' width=1000 align='center'><br/>

In [4]:
print('Create an VGG16 model along with the imagenet pre-trained weights.')
input_shape = (DatasetConfig.IMG_HEIGHT, DatasetConfig.IMG_WIDTH, DatasetConfig.CHANNELS)
vgg16_model = tf.keras.applications.vgg16.VGG16(input_shape=input_shape,
                                                include_top=True,
                                                weights='imagenet',
                                               )

Create an VGG16 model along with the imagenet pre-trained weights.


2023-04-24 10:50:18.687724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
print(vgg16_model.summary())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### 3.2 Inspect the original VGG-16 model

In [6]:
from tensorflow_model_optimization.quantization.keras import vitis_inspect

In [7]:
!cat ./arch/B512/arch-b512-lr.json

{"fingerprint":"0x101000016010200"}


In [8]:
inspector = vitis_inspect.VitisInspector(target="0x101000016010200")

In [9]:
inspected_model = inspector.inspect_model(vgg16_model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105025-513856_hvxJKr.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105025-513856_hvxJKr.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105025-513856_hvxJKr.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 4/4 [00:00<00:00, 2969.94it/s]                  
I20230424 10:50:26.729082  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:50:26.729112  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:50:26.729127  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:50:26.729360  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model, with op num: 4
I20230424 10:50:26.729369  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:50:26.733847  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:50:26.733878  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105025-513856_hvxJKr.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105025-513856_hvxJKr.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105025-513856_hvxJKr.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105025-513856_hvxJKr.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 4/4 [00:00<00:00, 1664.41it/s]                  

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105025-513856_hvxJKr.xmodel


...

[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105025-516168_WgeBub.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105025-516168_WgeBub.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105025-516168_WgeBub.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 6/6 [00:00<00:00, 27.88it/s]                    
I20230424 10:50:49.870944  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:50:49.870965  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:50:49.870970  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:50:49.871045  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_20, with op num: 8
I20230424 10:50:49.871050  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:50:50.110378  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:50:50.110404  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105025-516168_WgeBub.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105025-516410_OnpWVh.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105025-516410_OnpWVh.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105025-516410_OnpWVh.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 3/3 [00:00<00:00, 3719.45it/s]                  
I20230424 10:50:50.183846  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:50:50.183874  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:50:50.183884  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:50:50.184046  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_21, with op num: 3
I20230424 10:50:50.184054  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:50:50.188170  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:50:50.188210  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105025-516410_OnpWVh.xmodel
[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: vgg16
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/22        input_1                 InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/22        block1_conv1            Conv2D<relu>            DPU                                                         
-------------------------------------------------------------------------------------------

## 4 Inspecting the Modified Model


### 4.1 VGG Convolutional Base
We begin by creating a model of the VGG-16 convolutional base. We can do this by instantiating the model and setting `include_top = False`, which excludes the fully connected layers. In this notebook, we will instantiate the model with weights that were learned by training the model on the ImageNet dataset.

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/VGG16_10_pretrained_base.png' width=1000 align='center'><br/>

In [10]:
print('Create an VGG16 model along with the imagenet pre-trained weights.')
input_shape = (DatasetConfig.IMG_HEIGHT, DatasetConfig.IMG_WIDTH, DatasetConfig.CHANNELS)
vgg16_model_base = tf.keras.applications.vgg16.VGG16(input_shape=input_shape,
                                                    include_top=False,
                                                    weights='imagenet',
                                                   )

Create an VGG16 model along with the imagenet pre-trained weights.


In [11]:
print(vgg16_model_base.summary())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### 4.2 Add the Classification Layer (attempt 1)
Since we intend to train and use the model to classify hand signals from the ASL dataset (which has 29 classes), we will need to add our own classification layer. In this example, we have chosen to use just a single fully connected dense layer that contains 256 nodes, followed by a softmax output layer that contains 29 nodes for each of the 29 classes. The number of dense layers and the number of nodes per layer is a design choice, but the number of nodes in the output layer must match the number of classes in the dataset.

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/VGG16_02_asl.png' width=1000 align='center'><br/>

In [12]:
print('Add Classification Layers (output or top)')
# Take the output of the VGG16 just before last layer
x = vgg16_model_base.layers[-1].output

# Flatten the outputs of the last conv layer
flatten = tf.keras.layers.Flatten(name='flatten')(x)

# Fully Connected layers.
dense = tf.keras.layers.Dense(256, activation= 'relu',name='fc')(flatten)

# adding the prediction layer with 'softmax'
predictions = tf.keras.layers.Dense(DatasetConfig.NUM_CLASSES, activation='softmax',name='predictions')(dense)
    
# Create the new model with the new outputs
model = tf.keras.Model(inputs=vgg16_model_base.input, outputs=predictions)

Add Classification Layers (output or top)


In [13]:
print(model.summary())

Model: "model_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

### 4.3 Inspect the Modified Model (attempt 1)

In [14]:
inspected_model = inspector.inspect_model(model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105055-142572_MlSUDs.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105055-142572_MlSUDs.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105055-142572_MlSUDs.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 4/4 [00:00<00:00, 2125.31it/s]                  
I20230424 10:50:55.380667  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:50:55.380702  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:50:55.380712  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:50:55.380802  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_23, with op num: 4
I20230424 10:50:55.380815  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:50:55.385149  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:50:55.385182  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105055-142572_MlSUDs.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105055-142572_MlSUDs.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105055-142572_MlSUDs.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105055-142572_MlSUDs.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 4/4 [00:00<00:00, 2385.84it/s]                  
I20230424 10:50:55.473963  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:50:55.473989  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:50:55.473997  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:50:55.474061  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_24, with op num: 4
I20230424 10:50:55.474066  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:50:55.476984  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:50:55.477005  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105055-142572_MlSUDs.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105055-142572_MlSUDs.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105055-142572_MlSUDs.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105055-142572_MlSUDs.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


...

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105055-144632_eHbTrR.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105055-144853_POuAfQ.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105055-144853_POuAfQ.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105055-144853_POuAfQ.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 3/3 [00:00<00:00, 2360.33it/s]                  

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105055-144853_POuAfQ.xmodel



I20230424 10:51:00.110710  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:51:00.110735  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:51:00.110744  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:51:00.110883  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_43, with op num: 3
I20230424 10:51:00.110891  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:51:00.114202  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:51:00.114245  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model_22
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/21        input_23                InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/21        block1_conv1            Conv2D<relu>            DPU                                                         
------------------------------------------------------------------------------------------------------------------------
2/21        block1_conv2            Conv2D<relu>            DPU                           

### 4.4 Add the Classification Layer (attempt 2)
Since we intend to train and use the model to classify hand signals from the ASL dataset (which has 29 classes), we will need to add our own classification layer. In this example, we have chosen to use just a single fully connected dense layer that contains 256 nodes, followed by a softmax output layer that contains 29 nodes for each of the 29 classes. The number of dense layers and the number of nodes per layer is a design choice, but the number of nodes in the output layer must match the number of classes in the dataset.

<img src='https://github.com/AlbertaBeef/asl_tutorial/raw/2022.2/images/VGG16_03_asl_vitis_ai.png' width=1000 align='center'><br/>

In [15]:
print('Add Classification Layers (output or top)')
# Take the output of the VGG16 just before last layer
x = vgg16_model_base.layers[-1].output

# reduce the size of the last layer (7x7x512=25088)
# to account for conv2d/dense limitations in Vitis-AI
#x = tf.keras.layers.Conv2D(1024, (7, 7),name='block6_conv')(x)
#x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.MaxPooling2D()(x)
    
# Flatten the outputs of the last conv layer
flatten = tf.keras.layers.Flatten(name='flatten')(x)

# Fully Connected layers.
dense = tf.keras.layers.Dense(256, activation= 'relu',name='fc')(flatten)

# adding the prediction layer with 'softmax'
predictions = tf.keras.layers.Dense(DatasetConfig.NUM_CLASSES, activation='softmax',name='predictions')(dense)
    
# Create the new model with the new outputs
model = tf.keras.Model(inputs=vgg16_model_base.input, outputs=predictions)

Add Classification Layers (output or top)


In [16]:
print(model.summary())

Model: "model_44"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0  

### 4.5 Inspect the Modified Model (attempt 2)

In [17]:
inspected_model = inspector.inspect_model(model)

[VAI INFO] Update include_bias_corr: False
[VAI INFO] Update include_fast_ft: False
[VAI INFO] Update include_cle: False
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105103-332241_PvSQKz.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105103-332241_PvSQKz.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105103-332241_PvSQKz.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 4/4 [00:00<00:00, 4132.32it/s]                  
I20230424 10:51:03.567304  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:51:03.567344  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:51:03.567359  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:51:03.567471  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_45, with op num: 4
I20230424 10:51:03.567487  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:51:03.575191  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:51:03.575230  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105103-332241_PvSQKz.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105103-332241_PvSQKz.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105103-332241_PvSQKz.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105103-332241_PvSQKz.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 4/4 [00:00<00:00, 5195.79it/s]                  
I20230424 10:51:03.654947  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:51:03.654968  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:51:03.654973  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:51:03.655035  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_46, with op num: 4
I20230424 10:51:03.655041  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:51:03.658493  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:51:03.658522  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105103-332241_PvSQKz.xmodel
[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105103-332241_PvSQKz.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105103-332241_PvSQKz.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105103-332241_PvSQKz.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


...

[INFO] Namespace(batchsize=1, inputs_shape=None, layout='NHWC', model_files=['/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105103-333162_wUqYuE.h5'], model_type='tensorflow2', named_inputs_shape=None, out_filename='/tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105103-333162_wUqYuE.xmodel', proto=None)
[INFO] tensorflow2 model: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_quantized_20230424-105103-333162_wUqYuE.h5
[INFO] keras version: 2.10.0
[INFO] Tensorflow Keras model type: functional


[INFO] generate xmodel     :100%|██████████████████████████████████| 3/3 [00:00<00:00, 2826.35it/s]                  

[INFO] dump xmodel: /tmp/albertabeef/tensorflow_model_optimization/tmp_sub_graph_xnnc_20230424-105103-333162_wUqYuE.xmodel



I20230424 10:51:07.807456  5824 compile_pass_manager.cpp:287] [UNILOG][INFO] Compile mode: dpu
I20230424 10:51:07.807482  5824 compile_pass_manager.cpp:288] [UNILOG][INFO] Debug mode: null
I20230424 10:51:07.807490  5824 compile_pass_manager.cpp:292] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B512_0101000016010200
I20230424 10:51:07.807668  5824 compile_pass_manager.cpp:357] [UNILOG][INFO] Graph name: model_66, with op num: 3
I20230424 10:51:07.807678  5824 compile_pass_manager.cpp:370] [UNILOG][INFO] Begin to compile...
I20230424 10:51:07.812220  5824 compile_pass_manager.cpp:381] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20230424 10:51:07.812273  5824 compile_pass_manager.cpp:386] [UNILOG][INFO] Compile done.


[VAI INFO] Inspect Results:
[MODEL INFO]:
________________________________________________________________________________________________________________________
Model Name: model_44
________________________________________________________________________________________________________________________
ID          Name                    Type                    Device      Notes                                           
0/22        input_23                InputLayer              INPUT                                                       
------------------------------------------------------------------------------------------------------------------------
1/22        block1_conv1            Conv2D<relu>            DPU                                                         
------------------------------------------------------------------------------------------------------------------------
2/22        block1_conv2            Conv2D<relu>            DPU                           

## 5 Conclusion

In this notebook, we showed how you can use Vitis-AI's **model-inspector** to ensure compatibility with the DPU architecture.  Performing this step PRIOR to training saves a lot of pain and time. 